<a href="https://colab.research.google.com/github/araoli/APP/blob/main/scrapingiprhm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Autora: Patricia Araujo de Oliveira
#!pip install unidecode

import requests
import csv
from unidecode import unidecode
from bs4 import BeautifulSoup
import sqlite3

male = 0
female = 0
unknown = 0

all_num_author = 0
all_male = 0
all_female = 0
all_unknown = 0

all_first_author_male = 0
all_first_author_female = 0
all_first_author_unknown = 0

all_num_author_congresso = 0
all_male_congresso = 0
all_female_congresso = 0
all_unknown_congresso = 0

num_author_all_congresso = 0
male_all_congresso = 0
female_all_congresso = 0
unknown_all_congresso = 0


def extract_event_info(url):
  try:
    # Faz a requisição HTTP para a página dos anais
    response = requests.get(url)
    # Verifica se a requisição foi bem sucedida
    if response.status_code == 200:
      # Faz o parsing do HTML da página
      soup = BeautifulSoup(response.text, 'html.parser')

      # Encontra o elemento que contém o nome do evento
      event_name_element = soup.find('h1')
      event_name = event_name_element.text.strip()
      #print(event_name)

      #author_elements = []
      # Encontra os elementos que contêm os nomes dos autores
      # if author_elements == soup.find_all('div', class_="authors"):
      author_elements = soup.find_all('div', class_="authors")
      authors = [author.text.strip() for author in author_elements]

      if 'Autores' in authors[0]: authors.pop(0)
      #else: authors = ['']

      return event_name, authors

    else:
      print("Erro ao acessar a página dos anais da CSBC:",
            response.status_code)
      return []
  except Exception as e:
    print(
        "#Ocorreu um erro durante a extração dos nomes dos(as) autores(as) do evento",
        event_name, ":", e)
    return []


def extract_event_urls(url):
  event_urls = []

  # Realiza a requisição HTTP para obter o conteúdo da página
  response = requests.get(url)

  # Verifica se a requisição foi bem-sucedida
  if response.status_code == 200:
    # Faz o parsing do HTML da página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontra o elemento que contém o nome do evento
    congress_name_element = soup.find('h1')
    #congress_name_element = soup.find('div', class_="current")
    congress_name = congress_name_element.text.strip()

    # Encontra todas as tags 'li' com classe 'show'
    event_list_items = soup.find_all('li', class_='show')

    # Itera sobre os itens da lista de eventos encontrados
    for event_item in event_list_items:
      # Encontra o link dentro do item da lista e extrai a URL
      event_link = event_item.find('a')['href']

      # Adiciona a URL à lista de URLs de eventos
      event_urls.append(event_link)

  return event_urls, congress_name


def extract_congress_urls(url):
  congress_urls = []

  # Realiza a requisição HTTP para obter o conteúdo da página
  response = requests.get(url)

  # Verifica se a requisição foi bem-sucedida
  if response.status_code == 200:
    # Faz o parsing do HTML da página
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontra todas as tags 'li' com classe 'show'
    congress_list_items = soup.find_all('div', class_='obj_issue_summary')

    # Itera sobre os itens da lista de eventos encontrados
    for congress_item in congress_list_items:
      # Encontra o link dentro do item da lista e extrai a URL
      congress_link = congress_item.find('a')['href']

      # Adiciona a URL à lista de URLs de eventos
      congress_urls.append(congress_link)

  return congress_urls


# Função para consultar o banco de dados e obter o gênero
def check_gender(name):
  name_ajustado = unidecode(name).upper()

  conn = sqlite3.connect('grupos.db')
  cursor = conn.cursor()

  cursor.execute("SELECT classification FROM 'grupos' WHERE name=?",
                 (name_ajustado, ))
  result = cursor.fetchone()
  conn.close()

  if result is not None and len(result) > 0:
    return result[0]
  else:
    return 'Desconhecido'


def insert_into_db(name, gender):
  name_adjusted = unidecode(name).upper()
  try:
    conn = sqlite3.connect('grupos.db')
    cursor = conn.cursor()

    cursor.execute("INSERT INTO grupos (name, classification) VALUES (?, ?)",
                   (name_adjusted, gender))

    conn.commit()
    conn.close()

    print("Inserção bem-sucedida na tabela 'grupos'.")
  except sqlite3.Error as e:
    print("Erro ao inserir dados na tabela 'grupos':", e)


##########################################################################

# Conectar ao banco de dados SQLite
conn = sqlite3.connect('grupos.db')
cursor = conn.cursor()

# Fechar a conexão com o banco de dados
conn.close()

#url = 'https://sol.sbc.org.br/index.php/wit/issue/archive' #WIT
#url = 'https://sol.sbc.org.br/index.php/ctd/issue/archive' #CTD
#url = 'https://sol.sbc.org.br/journals/index.php/reic/issue/archive' #CTIC
#url = 'https://sol.sbc.org.br/index.php/semish/issue/archive' #SEMISH
#url = 'https://sol.sbc.org.br/index.php/wei/issue/archive' #WEI
#url = 'https://sol.sbc.org.br/index.php/brasnam/issue/archive' #BRASNAM
#url = 'https://sol.sbc.org.br/index.php/bwaif/issue/archive' #(BWAIF)
#url = 'https://sol.sbc.org.br/index.php/encompif/issue/archive' #ENCOMPIF
#url = 'https://sol.sbc.org.br/index.php/etc/issue/archive' #ETC
#url = 'https://sol.sbc.org.br/index.php/sbcup/issue/archive' #SBCUP
#url = 'https://sol.sbc.org.br/index.php/washes/issue/archive' #WASHES
#url = 'https://sol.sbc.org.br/index.php/wbl/issue/archive' #WBL
#url = 'https://sol.sbc.org.br/index.php/wcama/issue/archive' #WCAMA
#url = 'https://sol.sbc.org.br/index.php/wcge/issue/archive' #WCGE
#url = 'https://sol.sbc.org.br/index.php/wics/issue/archive' #WICS
#url = 'https://sol.sbc.org.br/index.php/wperformance/issue/archive' #WPERFORMANCE
#url = 'https://sol.sbc.org.br/index.php/wtestbeds/issue/archive' #WTESTBEDS
#url = 'https://sol.sbc.org.br/index.php/wtrans/issue/archive' #WTRANS
#url = 'https://sol.sbc.org.br/index.php/wbci/issue/archive' #WBCI
#url = 'https://sol.sbc.org.br/index.php/wfibre/issue/archive' #WFIBRE'
#url = 'https://sol.sbc.org.br/index.php/desafie/issue/archive' #DESFIE
#url = 'https://sol.sbc.org.br/index.php/sbsc/issue/archive' #SBSC
#url = 'https://sol.sbc.org.br/index.php/sbcas/issue/archive' #SBCAS
url = 'https://sol.sbc.org.br/index.php/bresci/issue/archive' #BRESCI


congress_urls = extract_congress_urls(url)

#print (congress_urls)

#for url_event in congress_urls:
 # print(url_event)


# URL da página que contém os eventos
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/1114' #2023
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/922' #2022
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/750' #2021
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/573' #2020
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/399' #2019
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/386' #2018
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/142' #2017
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/143' #2016
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/524' #2015
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/528' #2014
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/791' #2013
#url_event = 'https://sol.sbc.org.br/index.php/csbc/issue/view/793' #2012

  #event_urls, congress_name = extract_event_urls(url_event)
 # print(event_urls)

  #for evento in event_urls:
  #for evento in url_event:
for evento in congress_urls:

  event_info = extract_event_info(evento)

  if not event_info: continue

  event_name, author_names_uf = event_info
  #print(event_name)
  #print(author_names_uf)

  if author_names_uf:

    # Usando lista de compreensão para eliminar strings vazias e None
    author_names = [item for item in author_names_uf if item != ""]

    # Varre a lista de autores por artigo
    for indice, author_name in enumerate(author_names):

      author_list_paper = author_name.split(',')
      aux_author_list_paper = author_list_paper

      first_author = aux_author_list_paper[0].split()[0]

      #name_parts = author_name.split(',')
      #first_name = name_parts[0].split()[0]
      gender_first_author = check_gender(first_author)

      # Verifica o gênero do(a) primeiro(a) autor(a) por artigo
      if gender_first_author == 'Desconhecido':
        new_gender = input(
            f"Qual é o gênero de {first_author}? (M/F/D): ").upper()
        while new_gender not in ['M', 'F', 'D']:
          new_gender = input(
              "Por favor, insira M para masculino, F para feminino ou D para Desconhecido: "
          ).upper()
        insert_into_db(first_author, new_gender)
        gender_first_author = new_gender

      if gender_first_author == 'M':
        #male += 1
        all_male += 1
        all_first_author_male += 1
      elif gender_first_author == 'F':
        #female += 1
        all_female += 1
        all_first_author_female += 1
      else:
        #unknown += 1
        all_unknown += 1
        all_first_author_unknown += 1

      #print("----------------------------------")
      #print("## Artigo", indice+1, "##")

      # Soma de autores por artigo
      num_author_paper = len(author_list_paper)
      #print("Número de autores do ## Artigo", indice+1, "##", "=",  num_author_paper)

      # Soma de autores de todos os artigos
      all_num_author += num_author_paper

      aux_author_list_paper.pop(0)

      # Verifica autores homens e mulheres por artigo
      for author_first_name in aux_author_list_paper:
        #print(author_first_name.split()[0])
        first_name = author_first_name.split()[0]
        gender_author = check_gender(first_name)

        if gender_author == 'Desconhecido':
          new_gender = input(
              f"Qual é o gênero de {first_name}? (M/F/D): ").upper()
          while new_gender not in ['M', 'F', 'D']:
            new_gender = input(
                "Por favor, insira M para masculino, F para feminino ou D para Desconhecido: "
            ).upper()
          insert_into_db(first_name, new_gender)
          gender_author = new_gender

        if gender_author == 'M':
          #male += 1
          all_male += 1
        elif gender_author == "F":
          #female += 1
          all_female += 1
        else:
          #unknown += 1
          all_unknown += 1

      #print("Número de Autores Homens =", male)
    # print("Número de Autoras Mulheres =", female)
    #print("Número de Desconhecido =", unknown)
    # print("Gênero Primeiro(a) Autor(a) =", gender_first_author)
    #male = 0
    #female = 0
    #unknown = 0

    acuracia_first_author = (all_first_author_male +
                             all_first_author_female) / (len(author_names))
    acuracia_evento = (all_male + all_female) / all_num_author

    print("\n##############RESUMO EVENTO##############")

    # Imprime o nome do evento e os nomes dos autores
    print("Nome do Evento =", event_name)
   # print("Número de artigos =", len(author_names))
   # print("Total Primeiros Autores Homens =", all_first_author_male)
   # print("Total Primeiras Autoras Mulheres =", all_first_author_female)
   # print("Total Primeiros(as) Autores(as) Desconhecidos =",
   #       all_first_author_unknown)
   # print("Acurácia =", round(acuracia_first_author, 2))
  #  print("----------------------------------")
  #  print("Total de Autores(as) =", all_num_author)
   # print("Total de Autores Homens =", all_male)
  #  print("Total de Autoras Mulheres =", all_female)
  #  print("Total de Gênero Desconhecido =", all_unknown)
   # print("Acurácia =", round(acuracia_evento, 2))
    print("----------------------------------")

    # Cálculo de Índice de Presença Relativa de Homens e Mulheres (IPRHM)
    # IPRHM= [(M-H)/(M+H)]+ 1
    if all_female + all_male > 0:
      iprhm = ((all_female - all_male) / (all_female + all_male)) + 1
      print("IPRHM:", round(iprhm, 1))
      # Diagnóstico
      print("Diagnóstico:")
      if iprhm < 0.8:
        print("Desequilíbrio de Gênero com predominância MASCULINA")
      elif iprhm > 1.2:
        print("Desequilíbrio de Gênero com predominância FEMININA")
      else:
        print("IGUALDADE DE GÊNERO")
    else:
      print("Não há informações suficientes para calcular o IPRHM.")
    print("----------------------------------")

    all_num_author_congresso += all_num_author
    all_male_congresso += all_male
    all_female_congresso += all_female
    all_unknown_congresso += all_unknown

    all_num_author = 0
    all_male = 0
    all_female = 0
    all_unknown = 0
    all_first_author_male = 0
    all_first_author_female = 0
    all_first_author_unknown = 0

  else:

    print("\n##############RESUMO EVENTO##############")

    print("Evento", event_name,
          "NÃO POSSUI INFORMAÇÕES SOBRE OS(AS) AUTORES(AS)")
'''
num_author_all_congresso += all_num_author_congresso
male_all_congresso += all_male_congresso
female_all_congresso += all_female_congresso
unknown_all_congresso += all_unknown_congresso


all_num_author_congresso = 0
all_male_congresso = 0
all_female_congresso = 0
all_unknown_congresso = 0
'''

'''
acuracia_congresso = (all_male_congresso +
                      all_female_congresso) / all_num_author_congresso

print("\n##############CONGRESSO##############")

# Imprime informações sobre o evento
print("Congresso =", congress_name)
print("Total Autores(as) =", all_num_author_congresso)
print("Total Autores Homens =", all_male_congresso)
print("Total Autoras Mulheres =", all_female_congresso)
print("Total Autores(as) Desconhecidos =", all_unknown_congresso)
print("Acurácia =", round(acuracia_congresso, 2))
print("----------------------------------")
# Cálculo de Índice de Presença Relativa de Homens e Mulheres (IPRHM)
# IPRHM= [(M-H)/(M+H)]+ 1
if all_female_congresso + all_male_congresso > 0:
  iprhm = ((all_female_congresso - all_male_congresso) /
           (all_female_congresso + all_male_congresso)) + 1
  print("IPRHM:", round(iprhm, 1))
  # Diagnóstico
  print("Diagnóstico:")
  if iprhm < 0.8:
    print("Desequilíbrio de Gênero com predominância MASCULINA")
  elif iprhm > 1.2:
    print("Desequilíbrio de Gênero com predominância FEMININA")
  else:
    print("IGUALDADE DE GÊNERO")
else:
  print("Não há informações suficientes para calcular o IPRHM.")
print("----------------------------------")
'''
'''
print("\n##############HISTÓRICO##############")

# Imprime informações sobre o evento
print("Total Autores(as) =", num_author_all_congresso)
print("Total Autores Homens =", male_all_congresso)
print("Total Autoras Mulheres =", female_all_congresso)
print("Total Autores(as) Desconhecidos =", unknown_all_congresso)
print("----------------------------------")
# Cálculo de Índice de Presença Relativa de Homens e Mulheres (IPRHM)
# IPRHM= [(M-H)/(M+H)]+ 1
if female_all_congresso + male_all_congresso > 0:
    iprhm = ((female_all_congresso - male_all_congresso) / (female_all_congresso +
                                                           male_all_congresso)) + 1
    print("IPRHM:", round(iprhm, 2))
    # Diagnóstico
    print("Diagnóstico:")
    if iprhm < 0.8:
        print("Desequilíbrio de Gênero com predominância MASCULINA")
    elif iprhm > 1.2:
        print("Desequilíbrio de Gênero com predominância FEMININA")
    else:
        print("IGUALDADE DE GÊNERO")
else:
    print("Não há informações suficientes para calcular o IPRHM.")
print("----------------------------------")
'''


In [ ]:
!pip install unidecode